In [6]:
import spacy
import en_core_web_sm
import pandas as pd

df = pd.read_json('opiates_comments1.json')
display(df)
nlp = spacy.load("en_core_web_sm")
nlp = en_core_web_sm.load()
doc = nlp("This is a sentence.")
for token in doc:
    print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
            token.shape_, token.is_alpha, token.is_stop)






'body'

This this PRON DT nsubj Xxxx True True
is be AUX VBZ ROOT xx True True
a a DET DT det x True True
sentence sentence NOUN NN attr xxxx True False
. . PUNCT . punct . False False
